In [ ]:
from tqdm import tqdm
import subprocess
from sqlalchemy import create_engine

In [ ]:
# Informations d'identification PostgreSQL
db_user = "terence"
db_password = "Canon0-Spinout8-Debatable6-Proofread2"
db_host = "stockage-donnee-foncieres-gr02.postgres.database.azure.com"
db_port = 5432
db_name = "postgres"

In [ ]:
# Chaîne de connexion
conn_str = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

In [ ]:
# Créer un moteur SQLAlchemy
engine = create_engine(conn_str)
# Se connecter à la base de données
conn = engine.connect()

In [ ]:
import os

# Obtenir le chemin absolu du répertoire courant
current_dir = os.getcwd()

In [ ]:
import subprocess
from tqdm import tqdm

In [ ]:
# Fonction pour exécuter une ligne SQL
def execute_sql_line(line):
    try:
        # Exécuter la ligne SQL en tant que commande psql
        subprocess.run(["psql", "-U", "postgres", "-d", "postgres", "-c", line.strip()], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except subprocess.CalledProcessError as e:
        # Gérer les erreurs de commande
        print(f"Erreur lors de l'exécution de la commande : {e}")
    except Exception as e:
        # Gérer d'autres erreurs
        print(f"Une erreur inattendue s'est produite lors de l'exécution de la commande : {e}")


In [ ]:
# Fonction pour restaurer la base de données à partir du fichier SQL
def restore_database(sql_file_path_absolute):
    # Restauration de la base de données
    with tqdm(desc="Restauration de la base de données", unit="line") as pbar:
        try:
            # Ouvrir le fichier SQL en lecture
            with open(sql_file_path_absolute, "r") as f:
                # Lire le contenu du fichier ligne par ligne
                for line in f:
                    # Exécuter la ligne SQL
                    execute_sql_line(line)
                    # Mettre à jour la barre de progression
                    pbar.update(1)
        except FileNotFoundError:
            print("Le fichier SQL spécifié est introuvable.")
        except Exception as e:
            print(f"Une erreur inattendue s'est produite lors de la restauration de la base de données : {e}")



In [ ]:
# Chemin relatif vers le fichier SQL
sql_file_path_relative = "export_database.sql"
# Concaténer le chemin absolu du répertoire courant avec le chemin relatif du fichier SQL
sql_file_path_absolute = os.path.join(current_dir, sql_file_path_relative)

# Afficher le chemin absolu du fichier SQL
print("Chemin absolu du fichier SQL :", sql_file_path_absolute)

In [ ]:

# Utilisation de la fonction pour restaurer la base de données
restore_database(sql_file_path_absolute)

In [3]:
import re

In [8]:
# Chemin d'entrée et de sortie des fichiers
input_file = "./fichier_restauration.sql"
output_file = "restore_database.sql"


# Ouvrir les fichiers en lecture et en écriture
with open(input_file, "r") as f_input, open(output_file, "w") as f_output:
    # Indicateur pour suivre si nous devons traiter les lignes de données
    process_data = False
    
    # Lire chaque ligne du fichier d'entrée
    for line in f_input:
        # Vérifier si nous devons commencer à traiter les lignes de données
        if "COPY public.valeur_fonciere" in line:
            process_data = True
            f_output.write(line)  # Écrire la ligne originale dans le nouveau fichier
            continue
        
        # Vérifier si nous devons arrêter de traiter les lignes de données
        if process_data and line.strip() == "\\.":
            process_data = False
            f_output.write(line)  # Écrire la ligne originale dans le nouveau fichier
            continue
        
        # Si nous sommes en train de traiter les lignes de données, remplacer les tabulations par des virgules
        if process_data:
            line = line.replace("\t", ",")
        
        # Écrire les lignes dans le fichier de sortie
        f_output.write(line)